# Importing libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pymysql
from sqlalchemy import create_engine

# Requesting a Wikipedia page to scrape the table of Education Index

In [2]:
url = 'https://en.wikipedia.org/wiki/Education_Index'

In [3]:
# Making the request
html = requests.get(url).text

# Creating a BeautifulSoup instance, so we can search html tags
soup = BeautifulSoup(html, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Education Index - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XdlAPwpAIC8AABEM5FsAAAAV","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Education_Index","wgTitle":"Education Index","wgCurRevisionId":926399753,"wgRevisionId":926399753,"wgArticleId":20907300,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: archived copy as title","Democracy","Human Devel

In [4]:
# Searching for the tag table with the class wikitable
table = soup.find('table',{'class':'wikitable'});

In [5]:
table

<table class="wikitable sortable mw-collapsible" style="text-align:right;">
<caption>2015 Education Index<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[5]</a></sup>
</caption>
<tbody><tr>
<th>Rank</th>
<th>Country</th>
<th>Education<br/>Index</th>
<th>Expected years<br/>of schooling</th>
<th>Mean years<br/> of schooling</th>
<th>HDI rank</th>
<th>Continent
</th></tr>
<tr>
<td>1</td>
<td align="left"><span class="flagicon"><img alt="" class="thumbborder" data-file-height="640" data-file-width="1280" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/8/88/Flag_of_Australia_%28converted%29.svg/23px-Flag_of_Australia_%28converted%29.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/8/88/Flag_of_Australia_%28converted%29.svg/35px-Flag_of_Australia_%28converted%29.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/8/88/Flag_of_Australia_%28converted%29.svg/46px-Flag_of_Australia_%28converted%29.svg.png 2x" width="23"/> </s

In [6]:
# Finding all the tags 'th' (headers) within the html.
columns = table.findAll('th')

In [7]:
# Finding all the tags 'td' (cells) within the html.
cells = table.findAll('td')

In [8]:
columns

[<th>Rank</th>,
 <th>Country</th>,
 <th>Education<br/>Index</th>,
 <th>Expected years<br/>of schooling</th>,
 <th>Mean years<br/> of schooling</th>,
 <th>HDI rank</th>,
 <th>Continent
 </th>]

In [9]:
# Converting each element of the columns into plain text (remove the html tags)
def_columns = []
for i in columns: 
    def_columns.append(i.text)

In [10]:
def_columns

['Rank',
 'Country',
 'EducationIndex',
 'Expected yearsof schooling',
 'Mean years of schooling',
 'HDI rank',
 'Continent\n']

In [11]:
# Deleting the line break from the column name.
def_columns[-1] = def_columns[-1].replace('\n', '')

In [12]:
cells

[<td>1</td>,
 <td align="left"><span class="flagicon"><img alt="" class="thumbborder" data-file-height="640" data-file-width="1280" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/8/88/Flag_of_Australia_%28converted%29.svg/23px-Flag_of_Australia_%28converted%29.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/8/88/Flag_of_Australia_%28converted%29.svg/35px-Flag_of_Australia_%28converted%29.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/8/88/Flag_of_Australia_%28converted%29.svg/46px-Flag_of_Australia_%28converted%29.svg.png 2x" width="23"/> </span><a href="/wiki/Australia" title="Australia">Australia</a></td>,
 <td>0.939</td>,
 <td>20.4</td>,
 <td>13.2</td>,
 <td>2</td>,
 <td>Oceania
 </td>,
 <td>2</td>,
 <td align="left"><span class="flagicon"><img alt="" class="thumbborder" data-file-height="560" data-file-width="740" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/9/9c/Flag_of_Denmark.svg/

In [13]:
# Converting each element of the cells into plain text (remove the html tags)
def_cells = []
for i in cells:
    def_cells.append(i.text)

In [14]:
len(def_cells)

1316

In [15]:
# Creating a nested list so we can group all the common elements of the table to build the DataFrame later.
df_cells = []
for i in range(0, len(def_cells), 7):
    df_cells.append(def_cells[i: i+7]) 
    

In [16]:
df_cells

[['1', '\xa0Australia', '0.939', '20.4', '13.2', '2', 'Oceania\n'],
 ['2', '\xa0Denmark', '0.923', '19.2', '12.7', '5', 'Europe\n'],
 ['3', '\xa0New Zealand', '0.917', '19.2', '12.5', '13', 'Oceania\n'],
 ['4', '\xa0Norway', '0.916', '17.7', '12.7', '1', 'Europe\n'],
 ['5', '\xa0Germany', '0.914', '17.1', '13.2', '4', 'Europe\n'],
 ['6', '\xa0Ireland', '0.910', '18.6', '12.3', '8', 'Europe\n'],
 ['7', '\xa0Iceland', '0.906', '19.0', '12.2', '9', 'Europe\n'],
 ['8', '\xa0United States', '0.900', '16.5', '13.2', '10', 'North America\n'],
 ['9', '\xa0Netherlands', '0.897', '18.1', '11.9', '7', 'Europe\n'],
 ['10', '\xa0United Kingdom', '0.896', '16.3', '13.3', '16', 'Europe\n'],
 ['11', '\xa0\xa0Switzerland', '0.891', '16.0', '13.4', '2', 'Europe\n'],
 ['12', '\xa0Canada', '0.890', '16.3', '13.1', '10', 'North America\n'],
 ['13', '\xa0Slovenia', '0.886', '17.3', '12.1', '25', 'Europe\n'],
 ['14', '\xa0Lithuania', '0.882', '16.5', '12.7', '37', 'Europe\n'],
 ['15', '\xa0Czech Republic', '

In [17]:
# Creating the DataGrame
education_index = pd.DataFrame(df_cells, columns=def_columns)

In [18]:
# Testing how the DataFrame would be if we don't pass nested lists.
pd.DataFrame(def_cells)

,0
0,1
1,Australia
2,0.939
3,20.4
4,13.2
5,2
6,Oceania\n
7,2
8,Denmark
9,0.923


In [19]:
# Removing the line break from the continent names.
education_index['Continent'] = education_index['Continent'].apply(lambda x: x.replace('\n', ''))

In [20]:
education_index

,Rank,Country,EducationIndex,Expected yearsof schooling,Mean years of schooling,HDI rank,Continent
0,1,Australia,0.939,20.4,13.2,2,Oceania
1,2,Denmark,0.923,19.2,12.7,5,Europe
2,3,New Zealand,0.917,19.2,12.5,13,Oceania
3,4,Norway,0.916,17.7,12.7,1,Europe
4,5,Germany,0.914,17.1,13.2,4,Europe
5,6,Ireland,0.910,18.6,12.3,8,Europe
6,7,Iceland,0.906,19.0,12.2,9,Europe
7,8,United States,0.900,16.5,13.2,10,North America
8,9,Netherlands,0.897,18.1,11.9,7,Europe
9,10,United Kingdom,0.896,16.3,13.3,16,Europe


In [22]:
driver = 'mysql+pymysql'
host = "34.90.32.189"
username = 'root'
password = '123456789'
db = "education1"
connection_string = f'{driver}://{username}:{password}@{host}/{db}'
connection=create_engine(connection_string)
education_index.to_sql("education1", con=connection, index=False)

ValueError: Table 'education1' already exists.

In [23]:
education_index

,Rank,Country,EducationIndex,Expected yearsof schooling,Mean years of schooling,HDI rank,Continent
0,1,Australia,0.939,20.4,13.2,2,Oceania
1,2,Denmark,0.923,19.2,12.7,5,Europe
2,3,New Zealand,0.917,19.2,12.5,13,Oceania
3,4,Norway,0.916,17.7,12.7,1,Europe
4,5,Germany,0.914,17.1,13.2,4,Europe
5,6,Ireland,0.910,18.6,12.3,8,Europe
6,7,Iceland,0.906,19.0,12.2,9,Europe
7,8,United States,0.900,16.5,13.2,10,North America
8,9,Netherlands,0.897,18.1,11.9,7,Europe
9,10,United Kingdom,0.896,16.3,13.3,16,Europe


In [26]:
education_index.to_csv =('/Users/andressalomferrer/Desktop/ironhack/Projects/Project-Week-3-Data-Thieves/your-project')